In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
        
clearall()

In [2]:
############################
######## LIBRAIRIES ########
############################
print('Importing Librairies...')
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import talib
import pyttsx3
engine = pyttsx3.init()
from numpy import loadtxt
from functools import reduce
import time
from scipy.signal import argrelextrema
from ipywidgets import *
import multiprocessing as mp

print('Librairies imported\n')
print('version fxcmpy :',fxcmpy.__version__)
engine.say("librairie loaded")
engine.runAndWait()
print('Prêt')

Importing Librairies...
Librairies imported

version fxcmpy : 1.2.6
Prêt


In [3]:
engine.say("Ignition du Lumberjack")
engine.runAndWait() 

##############################################
######## INITIALISATION DES VARIABLES ########
##############################################


# COMPTE TEST #

TOKEN = '039add1dbe2b344ed08c518a98c7a1796f7a33c2'
server = 'demo'
user_id = 'D261219574'
compte = '01215057'
password = '3555'


'''
# COMPTE DEV #
TOKEN = '79f83cbff13d296eb6d9b6c1ed6dccd768ef925a'
server = 'demo'
user_id = 'D261219577'
compte = '01215060'
password = '3877'
'''


'''
# COMPTE POUBELLE #
TOKEN = 'a66bf5c0ec732487f4b348baf6364d3ee8784935'
server = 'demo'
user_id = 'D261219576'
compte = '01215059'
password = '8029'
'''


# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'


_number = 100
_period = 'm1'
_param = 'param_1min.csv'


_size = 100
_target = 15
_sl = -30
_rsi_high = 65
_rsi_low = 35

_err_allowed = 10/100

df = pd.DataFrame()

engine.say("Stuff is prepared, ready to make money")
engine.runAndWait()
print('Variables initialisées')

Variables initialisées


In [4]:

############################
######## FONCTIONS ########
###########################


def conX():
    con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error',server=server)
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
    return(con)

def deconX():
    con = con.close()
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion non intérrompue'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion intérrompue'+col.Style.RESET_ALL)
    return()

def scrap_close(ticker,number = 21):
    data = con.get_candles(_ticker,period=_period,number=_number)
    df = pd.DataFrame(index=data.index)
    df['Close'] = (data['bidclose']+data['askclose'])/2
    return(df)

def peak_detect(price,order=10):
    max_idx = list(argrelextrema(price,np.greater,order=order)[0])
    min_idx = list(argrelextrema(price,np.less,order=order)[0])
    idx = max_idx + min_idx + [len(price)-1]
    idx.sort()
    _current_idx = idx[-5:]
    
    _start = min(_current_idx)
    _end = max(_current_idx)
    _current_pat = price[_current_idx]
    return _current_idx,_current_pat,_start,_end  
  
def is_gartley(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.618 - err_allowed,0.618 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.27 - err_allowed,1.618 + err_allowed])*abs(BC)
        
    
    if XA>0 and AB<0 and BC>0 and CD<0:
        
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.isnan
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        
               
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
        
            return -1
        else:
            return np.isnan
    else:
        return np.isnan
    
def is_butterfly(moves,err_allowed):  
    
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.786 - err_allowed,0.786 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,2.618 + err_allowed])*abs(BC)
        
    
    if XA>0 and AB<0 and BC>0 and CD<0:
        
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        
               
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
        
            return -1
        else:
            return np.isnan
    else:
        return np.isnan       
  
def is_crab(moves,err_allowed):  
    
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.382 - err_allowed,0.618 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([2.24 - err_allowed,3.618 + err_allowed])*abs(BC)
        
    
    if XA>0 and AB<0 and BC>0 and CD<0:
        
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        
               
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
        
            return -1
        else:
            return np.isnan
    else:
        return np.isnan           
 
def is_bat(moves,err_allowed):  
    
    XA=moves[0]
    AB=moves[1]
    BC=moves[2]
    CD=moves[3]
    
    AB_range = np.array([0.382 - err_allowed,0.5 + err_allowed])*abs(XA)
    BC_range = np.array([0.382 - err_allowed,0.886 + err_allowed])*abs(AB)
    CD_range = np.array([1.618 - err_allowed,2.618 + err_allowed])*abs(BC)
        
    
    if XA>0 and AB<0 and BC>0 and CD<0:
        
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
            return 1
        else:
            return np.NaN
        
    elif XA<0 and AB>0 and BC<0 and CD>0:
        
               
        if AB_range[0]<abs(AB)<AB_range[1] and BC_range[0]<abs(BC)<BC_range[1] and CD_range[0]<abs(CD)<CD_range[1]:
        
            return -1
        else:
            return np.isnan
    else:
        return np.isnan

def scan(_ticker):
        
    if '/' in _ticker:
        _name = _ticker.replace('/','')
    
    if '.' in _ticker:
        _name = _ticker.replace('.','')
    
    if '.' not in _ticker and '/' not in _ticker:
        _name = _ticker
    
    globals()['data_%s' %_name] = pd.DataFrame()
    globals()['data_%s' %_name] = scrap_close(ticker = _ticker,number = _number)
    #Find Extrema
    _pips = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    _rsi = talib.RSI(globals()['data_%s' %_name].Close, timeperiod=14)
    _upperband, _middleband, _lowerband = talib.BBANDS(globals()['data_%s' %_name].Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    
    _current_idx,_current_pat,_start,_end = peak_detect(globals()['data_%s' %_name].values)

    XA = _current_pat[1] - _current_pat[0]
    AB = _current_pat[2] - _current_pat[1]
    BC = _current_pat[3] - _current_pat[2]
    CD = _current_pat[4] - _current_pat[3]


    _moves=[XA,AB,BC,CD]

    _gartley = is_gartley(_moves,_err_allowed)
    _butterfly = is_butterfly(_moves,_err_allowed)
    _crab = is_crab(_moves,_err_allowed)
    _bat = is_bat(_moves,_err_allowed)

    HARMONICS = np.array([_gartley,_butterfly,_bat,_crab])
    LABELS = ['gartley','butterly','bat','crab']

    if np.any(HARMONICS==1) or np.any(HARMONICS==-1 ):
        param = pd.read_csv(_param)
        try:
            param = param.drop(['Unnamed: 0'],axis=1)
        except:
            pass
        _tp = float(param[param['_ticker']==_ticker]['_tp'])
        _sl = float(param[param['_ticker']==_ticker]['_sl'])
        _amount = int(param[param['_ticker']==_ticker]['_amount'])
        if con.is_subscribed(_ticker) == False:
            con.subscribe_market_data(_ticker)
        _ask = con.get_last_price(_ticker).Ask
        _bid = con.get_last_price(_ticker).Bid
        _spread = (_ask-_bid)/_bid
        for j in range(0, len(HARMONICS)):
                    
            if HARMONICS[j]==-1 and _ticker not in list(con.get_open_positions().currency) and _spread < 0.0002:
                print(col.Fore.RED,'SHORT ON', col.Fore.YELLOW,_ticker,col.Style.RESET_ALL)
                engine.say("We are shooting, I repeat, foxtrot, foxtrot! We go short, over.")
                engine.runAndWait()
                
                print('Bid :',_bid,'-- Ask :',_ask,'-- Spread :',_spread)

                p0os = con.open_trade(symbol=_ticker,\
                                        is_buy=False,\
                                            amount=_amount,\
                                            time_in_force='DAY',\
                                                order_type='AtMarket',\
                                                rate=0,\
                                                    is_in_pips=False,\
                                                        limit= (con.get_last_price(_ticker).Bid) * (1-_tp),\
                                                            stop=(con.get_last_price(_ticker).Bid) * (1+_sl),\
                                                                trailing_step=None)


            
            elif HARMONICS[j]==1 and _ticker not in list(con.get_open_positions().currency) and _spread < 0.0002:
                print(col.Fore.GREEN,'LONG ON', col.Fore.YELLOW,_ticker,col.Style.RESET_ALL)
                engine.say("We are shooting, I repeat, foxtrot, foxtrot! We go long, over.")
                engine.runAndWait()

                print('Bid :',_bid,'-- Ask :',_ask,'-- Spread :',_spread)
                
                p0os = con.open_trade(symbol=_ticker,\
                                        is_buy=True,\
                                            amount=_amount,\
                                            time_in_force='DAY',\
                                                order_type='AtMarket',\
                                                rate=0,\
                                                    is_in_pips=False,\
                                                        limit= (con.get_last_price(_ticker).Ask) * (1+_tp),\
                                                            stop=(con.get_last_price(_ticker).Ask) * (1-_sl),\
                                                                trailing_step=None)
        

    else:
        print('\r',col.Fore.BLUE,_ticker,col.Style.RESET_ALL,end='',flush=True)

    return
engine.say("functions initiated")
engine.runAndWait()
print('Functions initiated')

Functions initiated


In [5]:
#########################
### CONNEXION A L'API ###
#########################

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor for FOR_EX\nLumberJack Jyss 5780(c)')
print(col.Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(col.Fore.YELLOW,col.Back.BLUE,'--- HarmoGo!em  #v0.10 ---',col.Style.RESET_ALL)

print('')
engine.say(" Initialisation du Harmau Gaulem")
engine.say("Connexion du Gaulem hà la Péh e")
engine.runAndWait()

try:
    con.is_connected() == True
    
    engine.say("already Connected")
    engine.runAndWait()
    print(col.Fore.GREEN+'Connexion rétablie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
    print('')
    
except:
    try:
        con = conX()
        con.is_connected() == True
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
        engine.say("Connected")
        engine.runAndWait()
    except:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
        engine.say("Not Connected, sal rass de mor, pitun cé la merd")
        engine.say("vérifi ton internet, é relance le gaulèm")
        engine.runAndWait()
        print('')
        #os._exit(0)
        con = deconX()
        time.sleep(1)
        con = conX()

Global Optimized LumberJack Environment Motor for FOR_EX
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  --- HarmoGo!em  #v0.10 --- 

Connexion établie
Compte utilisé :  [1215057]
Connexion établie
Compte utilisé :  [1215057]


In [6]:
for infinite in range(72000000):
    TICKERS = con.get_instruments()
    for _ticker in TICKERS:
        try:
            scan(_ticker)
        except:
            continue
    print('\n\nLoop',infinite,'effectuée. ','Heure locale :',col.Fore.LIGHTGREEN_EX,dt.datetime.now().strftime("%H:%M:%S"),col.Style.RESET_ALL,'Pause\n\n')
    time.sleep(30)

 USD/ILS  LONG ON  USD/CNH 
Bid : 7.00213 -- Ask : 7.00265 -- Spread : 7.426311708006762e-05
  USEquities 

Loop 0 effectuée.  Heure locale :  08:33:16  Pause


  USEquities 

Loop 1 effectuée.  Heure locale :  08:34:00  Pause


  USEquities 

Loop 2 effectuée.  Heure locale :  08:34:46  Pause


  USEquities 

Loop 3 effectuée.  Heure locale :  08:35:30  Pause


  USEquities 

Loop 4 effectuée.  Heure locale :  08:36:14  Pause


  USEquities 

Loop 5 effectuée.  Heure locale :  08:37:03  Pause


  USEquities 

Loop 6 effectuée.  Heure locale :  08:37:47  Pause


  USEquities 

Loop 7 effectuée.  Heure locale :  08:38:31  Pause


  USEquities 

Loop 8 effectuée.  Heure locale :  08:39:19  Pause


  USEquities 

Loop 9 effectuée.  Heure locale :  08:40:02  Pause


  USEquities 

Loop 10 effectuée.  Heure locale :  08:40:51  Pause


  USEquities 

Loop 11 effectuée.  Heure locale :  08:41:35  Pause


  USEquities 

Loop 12 effectuée.  Heure locale :  08:42:19  Pause


  USEquities 

Loop 

KeyboardInterrupt: 

In [ ]:
##### NE PAS LANCER CI-DESSOUS MERCI ####

In [ ]:
TICKERS = con.get_instruments()
_cpu = mp.cpu_count()
print(_cpu)
pool = mp.Pool(processes=(_cpu-1))
pool.map(scan,TICKERS)
pool.terminate()
pool.join()